## Let's Ask Some Important and Interesting Questions Regarding Corona

- How is Corona growing Worldwide, Inside China, and the rest of the World?
- Country-wise infection as of today?
- How is the country-wise growth rate?

In [17]:
import numpy as np
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go

'''
Data source: Johns Hopkins University (CSSEGISandData) [Keeps updating everyday]
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
'''

confirmed = 'data/time_series_19-covid-Confirmed.csv'
recovered = 'data/time_series_19-covid-Recovered.csv'
death = 'data/time_series_19-covid-Deaths.csv'

# CONFIRMED cases => [World], [Mainland China], and [Rest of the world]
world_data_confirmed = pd.read_csv(confirmed)
mainland_china_data_confirmed = world_data_confirmed[world_data_confirmed['Country/Region'].str.contains("China")]
outside_china_data_confirmed = world_data_confirmed[~world_data_confirmed['Country/Region'].str.contains("China")]

# RECOVERED cases => [World], [Mainland China], and [Rest of the world]
world_data_recovered = pd.read_csv(recovered)
mainland_china_data_recovered = world_data_recovered[world_data_recovered['Country/Region'].str.contains("China")]
outside_china_data_recovered = world_data_recovered[~world_data_recovered['Country/Region'].str.contains("China")]

# DEATHS cases => [World], [Mainland China], and [Rest of the world]
world_data_death = pd.read_csv(confirmed)
mainland_china_data_death = world_data_death[world_data_death['Country/Region'].str.contains("China")]
outside_china_data_death = world_data_death[~world_data_death['Country/Region'].str.contains("China")]

In [18]:
def sum_column(df):
    if df.iloc[:,-1].isnull().values.any():
        df = df.iloc[:, :-1]
    sum = df.sum(axis=0)
    return sum[world_data_confirmed.columns[4:]].to_frame()


In [23]:
region = ['World', 'Mainland China', 'Outside China']

def plot_line(df, region_code):
    df_to_plot_world_confirm = sum_column(df)
    fig = px.line(df_to_plot_world_confirm, x=df_to_plot_world_confirm.index.values, y=df_to_plot_world_confirm[df_to_plot_world_confirm.columns[0]] )
    fig.update_layout(
        title="Day-By-Day " + region[region_code] + " Infected",
        xaxis_title="Days",
        yaxis_title="People Infected",
        font=dict(
            family="Arial",
            size=13,
            color="#7f7f7f"
        )
    )
    fig.show()

def plot_country_based_line(df):
    countries_as_of_today = df.groupby(['Country/Region']).sum().iloc[:,-1].to_frame()
    sorted_countries = countries_as_of_today.sort_values(by=[str(countries_as_of_today.columns.values[0])],ascending=False)
    fig = px.bar(
        sorted_countries, 
        x=sorted_countries.index.values, 
        y=sorted_countries[sorted_countries.columns[0]])
    fig.update_layout(
            title="Country Wise Infected",
            xaxis_title="Countries",
            yaxis_title="People Infected",
            font=dict(
                family="Arial",
                size=13,
                color="#7f7f7f"
            ))
    fig.show()

## Mainland China Day By Day
How is the infection spreading inside China?

In [24]:
plot_line(mainland_china_data_confirmed, 1)

## Whole World Day By Day
How is the infection spreading Worldwide?

In [5]:
plot_line(world_data_confirmed, 0)

## Outside China Day By Day
China seems to have contained the impacts of Coronavirus as of now. But how is the infection spreading outside China?

In [6]:
plot_line(outside_china_data_confirmed, 2)

## Country-wise as of today
How are the countries being infected?

In [7]:
plot_country_based_line(world_data_confirmed)

## Country-wise as of today (except China)?
How are countries except China being infected?

In [8]:
plot_country_based_line(outside_china_data_confirmed)

## Country-wise infection growth comparision

In [9]:
countries_as_of_today = outside_china_data_confirmed.groupby(['Country/Region']).sum()
sorted_outside_china = countries_as_of_today.sort_values(by=[countries_as_of_today.columns.values[-1]], ascending=False)
title = 'Infection outside China'
labels = sorted_outside_china.index.values.tolist()
x_data = np.vstack((sorted_outside_china.columns.values[2:],)*len(sorted_outside_china.index))
y_data = sorted_outside_china.iloc[:,2:len(sorted_outside_china.columns)].to_numpy()

fig = go.Figure()

for i in range(0, len(sorted_outside_china.index)):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',name=labels[i],connectgaps=True,))
    fig.add_trace(go.Scatter(x=[x_data[i][0], x_data[i][-1]],y=[y_data[i][0], y_data[i][-1]],mode='markers',))

fig.update_layout(xaxis=dict(showline=True,showgrid=False,showticklabels=True,linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []


# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Infection growth outside China',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Johns Hopkins University (CSSEGISandData) and WHO',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations,width=800,
    height=1000,)

fig.show()

Iran and Italy are still struggling to slow the growth rate and this can worsen.   
<b>Meanwhile, Germany, France, Spain, and the US are showing similar early trends like that of Iran and Italy.</b>

## More analysis